In [7]:
from BRAD import llms
llm = llms.load_nvidia()


In [8]:
import subprocess
import os
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
import chromadb
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [9]:
persist_directory = '/nfs/turbo/umms-indikar/shared/projects/RAG/databases/DigitalLibrary-10-June-2024/'
embeddings_model = HuggingFaceEmbeddings(model_name='BAAI/bge-base-en-v1.5')
db_name = "DigitalLibrary"
_client_settings = chromadb.PersistentClient(path=(persist_directory + db_name))
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings_model, client=_client_settings, collection_name=db_name)

/home/oliven/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
prompt = "What is Pip Fucci for the cell cycle?"
documentSearch = vectordb.similarity_search_with_relevance_scores(prompt, k=5)

In [11]:
documentSearch

[(Document(page_content='S phase is best visualized in cells with high reporter\nexpression since it requires accumulation above the\ndetection threshold.\nBecause DNA replication is an essentially irre-\nversible step, S phase entry and progression is\ntightly regulated. Detecting the G1/S transition in\nliving cells is not a simple technological task how-\never. PIP-FUCCI is a precise method for demar-\ncating cell cycle phase transitions in mammalian\ncells. Relative to first-generation FUCCI reporters,\nthe simplicity of the new PIP-mVenus fragment\nreduces the number of confounding factors that\nmust be considered when designing and interpret-\ning experiments. PIP-FUCCI reagents are available\nto researchers for straightforward cell cycle phase\ndetermination. We anticipate that the ability to\nclearly distinguish G1 from S phase and S phase\nfrom G2 in both fixed and live cells will be parti-\ncularly useful for investigations focused on cell\nproliferation commitment and the ma

In [12]:
for item in documentSearch:
    doc = item[0]
    source = doc.metadata.get('source')
    short_source = os.path.basename(source)
    print(f"Source: {short_source}")

Source: Accurate delineation of cell cycle phase transitions in living cells with PIP FUCCI.pdf
Source: Accurate delineation of cell cycle phase transitions in living cells with PIP FUCCI.pdf
Source: Accurate delineation of cell cycle phase transitions in living cells with PIP FUCCI.pdf
Source: Accurate delineation of cell cycle phase transitions in living cells with PIP FUCCI.pdf
Source: Accurate delineation of cell cycle phase transitions in living cells with PIP FUCCI.pdf


In [13]:
def queryDocs(chatstatus):
    """
    Queries documents based on the user prompt and updates the chat status with the results.

    :param chatstatus: A dictionary containing the current chat status, including the prompt, LLM, vector database, and memory.
    :type chatstatus: dict

    :raises KeyError: If required keys are not found in the chatstatus dictionary.
    :raises AttributeError: If methods on the vector database or LLM objects are called incorrectly.

    :return: The updated chat status dictionary with the query results.
    :rtype: dict
    """
    process = {}
    llm      = chatstatus['llm']              # get the llm
    prompt   = chatstatus['prompt']           # get the user prompt
    vectordb = chatstatus['databases']['RAG'] # get the vector database
    memory   = chatstatus['memory']           # get the memory of the model
    
    # query to database
    if vectordb is not None:
        documentSearch = vectordb.similarity_search_with_relevance_scores(prompt, k=chatstatus['config']['num_articles_retrieved'])
        docs, scores = getDocumentSimilarity(documentSearch)
        chain = load_qa_chain(llm, chain_type="stuff", verbose = chatstatus['config']['debug'])
        #bertscores
        if chatstatus['config']['experiment'] is True:
            # scoring_experiment(chain, docs, scores, prompt)
            crossValidationOfDocumentsExperiment(chain, docs, scores, prompt, chatstatus)
            chatstatus['process'] = {'type': 'docs cross validation experiment'}
        else:
            # pass the database output to the llm       
            res = chain({"input_documents": docs, "question": prompt})
            print(res['output_text'])
            for item in documentSearch:
                doc = item[0]
                source = doc.metadata.get('source')
                short_source = os.path.basename(source)
                print(f"Source: {short_source}")
            # change inputs to be json readable
            res['input_documents'] = getInputDocumentJSONs(res['input_documents'])
            chatstatus['output'], chatstatus['process'] = res['output_text'], res
    else:
        template = """Current conversation: {history}\n\n\nNew Input: \n{input}"""
        PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)
        conversation = ConversationChain(prompt  = PROMPT,
                                         llm     = llm,
                                         verbose = chatstatus['config']['debug'],
                                         memory  = memory,
                                        )
        prompt = getDefaultContext() + prompt
        response = conversation.predict(input=prompt)
        print(response)
        chatstatus['output'] = response
        chatstatus['process'] = {'type': 'LLM Conversation'}
    # update and return the chatstatus
    # chatstatus['output'], chatstatus['process'] = res['output_text'], res
    return chatstatus

In [1]:
from BRAD import brad


In [18]:
### ok yeah this shit aint workign (originally wanted it so i could make sure I'm putting the cited sources in the right place)
### so im gonna spend a while trying to figure out the cosine similarity stuff 

In [ ]:
brad.chat()


Welcome to RAG! The chat log from this conversation will be saved to /home/oliven/BRAD/2024-06-14-15:18:25.200414.json. How can I help?


Enter your NVIDIA API key:  ········



Would you like to use a database with BRAD [Y/N]?


 y


/home/oliven/.local/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/oliven/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Input >>  What is Pip Fucci for the cell cycle?


RAG >> 1: 

/home/oliven/.local/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
for long-term storage. PBAN, a Gateway-compatible, doxycycline-inducible, puromycin-selectable PiggyBac vector was constructed from PB-TRE-dCas9-VPR (Addgene no. 63800) and used to deliver TFs to hiPSCs using a Lonza Nucleofector X-Unit (AAF-1002X), following the manufacturer’s instructions. Electroporated cells were transferred to a six-well Matrigel-coated plate in mTeSR1 with ROCK inhibitor. Cells were selected with 1 μg ml−1 of puromycin (Gibco, A1113803).

Flow cytometry and FACS. For flow cytometry analysis and FACS, cells were dissociated using TrypLE Express, washed and resuspended in FACS buffer (PBS with 10% FBS). For surface antigens, live cells were stained with

for long-term storage. PB